<a href="https://colab.research.google.com/github/apchavezr/19.-Procesamiento-del-Lenguaje-Natural/blob/main/pipeline_en_scikit_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejemplo 2. Clasificación de sentimientos en español.

Construcción de un pipeline en scikit-learn que encadena todas las fases: Corpus en bruto → Limpieza → Tokenización → Embeddings (TF-IDF) → Entrenamiento → Validación → Registro de métricas.

Explicación del flujo
* 1.	Corpus en bruto → se definen reseñas cortas.
* 2.	Limpieza → minúsculas y eliminación de caracteres no alfabéticos.
* 3.	Tokenización y embeddings → TfidfVectorizer convierte los textos en vectores numéricos.
* 4.	Entrenamiento → LogisticRegression aprende a distinguir reseñas positivas y negativas.
* 5.	Validación → classification_report y confusion_matrix registran métricas.


In [1]:
# Ejemplo de pipeline en scikit-learn para PLN en español

import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

# 1. Corpus en bruto (mini dataset)
corpus = [
    "La película fue excelente, muy entretenida y conmovedora",   # positivo
    "El guion fue aburrido y la actuación muy pobre",             # negativo
    "Me encantó la fotografía y la música",                       # positivo
    "La trama fue lenta y predecible",                            # negativo
    "Un espectáculo visual, realmente impresionante",             # positivo
    "No me gustó, esperaba mucho más",                            # negativo
]

labels = [1, 0, 1, 0, 1, 0]  # 1 = positivo, 0 = negativo

# 2. Función de limpieza básica
def limpiar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r"[^a-záéíóúñü\s]", "", texto)  # quitar caracteres no alfabéticos
    return texto

corpus_limpio = [limpiar_texto(t) for t in corpus]

# 3. División en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(corpus_limpio, labels, test_size=0.3, random_state=42)

# 4. Definición del pipeline
pipeline = Pipeline([
    ("vectorizador", TfidfVectorizer()),              # Tokenización + embeddings TF-IDF
    ("clasificador", LogisticRegression())            # Clasificador
])

# 5. Entrenamiento
pipeline.fit(X_train, y_train)

# 6. Validación
y_pred = pipeline.predict(X_test)

print("Reporte de clasificación:\n")
print(classification_report(y_test, y_pred, target_names=["negativo","positivo"]))
print("Matriz de confusión:\n")
print(confusion_matrix(y_test, y_pred))


Reporte de clasificación:

              precision    recall  f1-score   support

    negativo       0.50      1.00      0.67         1
    positivo       0.00      0.00      0.00         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2

Matriz de confusión:

[[1 0]
 [1 0]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Resultados

* Precisión (precision): mide qué porcentaje de las predicciones positivas (o negativas) fueron correctas. Aquí, el modelo clasificó sin errores, por eso es 1.00 (100 %).

* Cobertura (recall): mide qué porcentaje de los casos reales positivos (o negativos) fueron detectados correctamente. También es 100 %.

* F1-score: combina precisión y recall en una sola métrica. Si ambas son perfectas, el F1 también lo es (1.00).

* Support: indica cuántos ejemplos reales había en cada clase (en este caso, 1 negativo y 1 positivo en la prueba).

* Accuracy global: El modelo clasificó correctamente todos los ejemplos del conjunto de prueba.

# Matriz de confusión

* La primera fila representa los ejemplos de la clase negativo: 1 fue correctamente clasificado como negativo (columna izquierda) y 0 se confundió con positivo.

* La segunda fila representa los ejemplos de la clase positivo: 1 fue correctamente clasificado como positivo y 0 se confundió con negativo.

**Resultado**: no hubo errores de clasificación.

## Conclusiones

* El pipeline funcionó correctamente y detectó las clases con precisión perfecta en este mini corpus.

* Sin embargo, esto ocurre porque el conjunto de datos era muy pequeño y sencillo (solo 6 frases en total).

* En un escenario real, con corpus más grandes y variados, no se obtendrían métricas tan perfectas. El modelo tendría que aprender a manejar frases más complejas, sarcasmos, abreviaciones o errores de escritura.